문제 2-2: 영화 리뷰 감정 분석기

In [5]:
import os
from enum import Enum
from pprint import pprint
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser

# .env 파일에서 환경 변수 로드
load_dotenv()

# LLM 설정 (Upstage Solar 모델 사용)
llm = ChatUpstage(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    model="solar-1-mini-chat", # solar-1-mini-chat-ko-instruct 모델도 사용 가능
    temperature=0.1
)
# 1. 감정 분류를 위한 Enum 클래스 정의
class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# 2. EnumOutputParser 생성
parser = EnumOutputParser(enum=Sentiment)

# 3. 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(
    """다음 영화 리뷰의 감정을 분석하여, 반드시 '긍정', '부정', '보통' 중 하나로만 답변해야 합니다. 
    다른 어떤 설명도 추가하지 마세요.
    
    {instructions}

    리뷰: {review}
    답변:"""
)

# 4. 체인 생성
chain = prompt | llm | parser

# 5. 여러 리뷰로 테스트
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
]

for i, review in enumerate(test_reviews):
    result = chain.invoke({
        "review": review,
        "instructions": parser.get_format_instructions(),
    })
    print(f"## 리뷰 {i+1}: \"{review}\"")
    print(f"-> 분석 결과: {result.value}\n")

## 리뷰 1: "이 영화 정말 재미없어요. 시간 낭비였습니다."
-> 분석 결과: 부정

## 리뷰 2: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
-> 분석 결과: 긍정

## 리뷰 3: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
-> 분석 결과: 보통

